In [ ]:
import pandas as pd
import re
import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

/Users/devin/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
train_df = pd.read_csv("../data/raw/train.csv")
test_X = pd.read_csv("../data/raw/test.csv")

### train test split

In [ ]:
SEED = 20010929

In [ ]:
train_X, val_X, train_Y, val_Y = train_test_split(train_df.drop(['Survived'], axis = 1), train_df['Survived'], test_size=0.2, random_state=42)

In [ ]:
train_Y

331    0
733    0
382    0
704    0
813    0
      ..
106    1
270    0
860    0
435    1
102    0
Name: Survived, Length: 712, dtype: int64

## Handle Missing Data

In [ ]:
for column in train_X.columns:
    if (train_X[column].isna().sum() > 0) or (val_X[column].isna().sum() > 0):
        print(column)

Age
Cabin
Embarked


#### Embarked

In [ ]:
train_X['Embarked'].isna().sum()

2

In [ ]:
train_X['Embarked'].value_counts()

Embarked
S    525
C    125
Q     60
Name: count, dtype: int64

In [ ]:
# fill null value with majority embarked city
train_X['Embarked'] = train_X['Embarked'].fillna('S')
val_X['Embarked'] = val_X['Embarked'].fillna('S')
test_X['Embarked'] = train_df['Embarked'].fillna('S')

#### Age

In [ ]:
train_X['Age'].isna().sum()

140

In [ ]:
train_X['Age'].describe()

count    572.000000
mean      29.498846
std       14.500059
min        0.420000
25%       21.000000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [ ]:
train_X['Age'] = train_X['Age'].fillna(train_X['Age'].median())
val_X['Age'] = val_X['Age'].fillna(val_X['Age'].median())
test_X['Age'] = test_X['Age'].fillna(test_X['Age'].median())

#### Cabin

In [ ]:
train_X['Cabin'].isna().sum()

553

In [ ]:
train_X['Cabin'].value_counts()

Cabin
C23 C25 C27    4
E101           3
B96 B98        3
C22 C26        3
G6             3
              ..
C104           1
D11            1
C86            1
C7             1
C62 C64        1
Name: count, Length: 117, dtype: int64

In [ ]:
imputed_train_X = train_X.drop('Cabin', axis=True)
imputed_val_X = val_X.drop('Cabin', axis=True)
imputed_test_X = test_X.drop('Cabin', axis=True)

In [ ]:
imputed_train_X

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
331,332,1,"Partner, Mr. Austen",male,45.5,0,0,113043,28.5000,S
733,734,2,"Berriman, Mr. William John",male,23.0,0,0,28425,13.0000,S
382,383,3,"Tikkanen, Mr. Juho",male,32.0,0,0,STON/O 2. 3101293,7.9250,S
704,705,3,"Hansen, Mr. Henrik Juul",male,26.0,1,0,350025,7.8542,S
813,814,3,"Andersson, Miss. Ebba Iris Alfrida",female,6.0,4,2,347082,31.2750,S
...,...,...,...,...,...,...,...,...,...,...
106,107,3,"Salkjelsvik, Miss. Anna Kristine",female,21.0,0,0,343120,7.6500,S
270,271,1,"Cairns, Mr. Alexander",male,28.0,0,0,113798,31.0000,S
860,861,3,"Hansen, Mr. Claus Peter",male,41.0,2,0,350026,14.1083,S
435,436,1,"Carter, Miss. Lucile Polk",female,14.0,1,2,113760,120.0000,S


### Fare

In [ ]:
for column in imputed_train_X.columns:
    if (imputed_test_X[column].isna().sum() > 0):
        print(column)

Fare


In [ ]:
imputed_test_X['Fare'] = imputed_test_X['Fare'].fillna(imputed_test_X['Fare'].median())

## Extract Title and Family Name from Name

In [ ]:
name_pattern = r'^(?P<family_name>.+),\s(?P<title>Mr\.|Mrs\.|Miss\.|Rev\.|Master\.|Dr\.|Mme\.|Major\.|Sir\.|Mlle\.|Col\.|Capt\.|Don\.|Ms\.|Lady\.|the\sCountess\.|Jonkheer\.)'

In [ ]:
def extract_name(name):
    match = re.search(name_pattern, name)
    if match:
        family_name = match.group('family_name')
        title = match.group('title')
        return pd.Series([family_name, title])
    return pd.Series([None, None])

In [ ]:
imputed_train_X[['family_name', 'title']] = imputed_train_X['Name'].apply(extract_name)
imputed_val_X[['family_name', 'title']] = imputed_val_X['Name'].apply(extract_name)
imputed_test_X[['family_name', 'title']] = imputed_test_X['Name'].apply(extract_name)

In [ ]:
imputed_train_X

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,family_name,title
331,332,1,"Partner, Mr. Austen",male,45.5,0,0,113043,28.5000,S,Partner,Mr.
733,734,2,"Berriman, Mr. William John",male,23.0,0,0,28425,13.0000,S,Berriman,Mr.
382,383,3,"Tikkanen, Mr. Juho",male,32.0,0,0,STON/O 2. 3101293,7.9250,S,Tikkanen,Mr.
704,705,3,"Hansen, Mr. Henrik Juul",male,26.0,1,0,350025,7.8542,S,Hansen,Mr.
813,814,3,"Andersson, Miss. Ebba Iris Alfrida",female,6.0,4,2,347082,31.2750,S,Andersson,Miss.
...,...,...,...,...,...,...,...,...,...,...,...,...
106,107,3,"Salkjelsvik, Miss. Anna Kristine",female,21.0,0,0,343120,7.6500,S,Salkjelsvik,Miss.
270,271,1,"Cairns, Mr. Alexander",male,28.0,0,0,113798,31.0000,S,Cairns,Mr.
860,861,3,"Hansen, Mr. Claus Peter",male,41.0,2,0,350026,14.1083,S,Hansen,Mr.
435,436,1,"Carter, Miss. Lucile Polk",female,14.0,1,2,113760,120.0000,S,Carter,Miss.


### Extract Ticket title and number from Ticket

In [ ]:
ticket_pattern = r'(?P<ticket_prefix>.*\D)?(?P<ticket_number>\d+)'

In [ ]:
def extract_ticket(ticket):
    match = re.search(ticket_pattern, ticket)
    if match:
        ticket_prefix = match.group('ticket_prefix')
        ticket_number = match.group('ticket_number')
        return pd.Series([ticket_prefix, ticket_number])
    return pd.Series([None, None])

In [ ]:
imputed_train_X[['ticket_prefix', 'ticket_number']] = imputed_train_X['Ticket'].apply(extract_ticket)
imputed_val_X[['ticket_prefix', 'ticket_number']] = imputed_val_X['Ticket'].apply(extract_ticket)
imputed_test_X[['ticket_prefix', 'ticket_number']] = imputed_test_X['Ticket'].apply(extract_ticket)

### 

In [ ]:
imputed_train_X

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,family_name,title,ticket_prefix,ticket_number
331,332,1,"Partner, Mr. Austen",male,45.5,0,0,113043,28.5000,S,Partner,Mr.,None,113043
733,734,2,"Berriman, Mr. William John",male,23.0,0,0,28425,13.0000,S,Berriman,Mr.,None,28425
382,383,3,"Tikkanen, Mr. Juho",male,32.0,0,0,STON/O 2. 3101293,7.9250,S,Tikkanen,Mr.,STON/O 2.,3101293
704,705,3,"Hansen, Mr. Henrik Juul",male,26.0,1,0,350025,7.8542,S,Hansen,Mr.,None,350025
813,814,3,"Andersson, Miss. Ebba Iris Alfrida",female,6.0,4,2,347082,31.2750,S,Andersson,Miss.,None,347082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,107,3,"Salkjelsvik, Miss. Anna Kristine",female,21.0,0,0,343120,7.6500,S,Salkjelsvik,Miss.,None,343120
270,271,1,"Cairns, Mr. Alexander",male,28.0,0,0,113798,31.0000,S,Cairns,Mr.,None,113798
860,861,3,"Hansen, Mr. Claus Peter",male,41.0,2,0,350026,14.1083,S,Hansen,Mr.,None,350026
435,436,1,"Carter, Miss. Lucile Polk",female,14.0,1,2,113760,120.0000,S,Carter,Miss.,None,113760


In [ ]:
imputed_train_X['ticket_prefix'].value_counts()

ticket_prefix
PC              42
C.A.            22
STON/O 2.       10
CA.              8
A/5.             7
A/5              7
SOTON/OQ         7
STON/O2.         6
W./C.            6
CA               6
SC/PARIS         5
C                5
SOTON/O.Q.       4
F.C.C.           4
A/4.             3
SC/Paris         3
S.O.C.           3
A.5.             2
WE/P             2
P/PP             2
S.C./PARIS       2
A./5.            2
S.O./P.P.        2
PP               2
A/4              2
SOTON/O2         1
C.A./SOTON       1
F.C.             1
W/C              1
S.O.P.           1
SW/PP            1
SC               1
SC/AH            1
S.C./A.4.        1
Fa               1
A/S              1
SC/AH Basle      1
W.E.P.           1
S.W./PP          1
SCO/W            1
A4.              1
Name: count, dtype: int64

### drop PassengerId and ticket number

In [ ]:
for feature in imputed_train_X.columns:
    if imputed_train_X[feature].isna().sum() > 0:
        print(feature, '\n')

ticket_prefix 

ticket_number 



In [ ]:
correlation_df = pd.DataFrame({
    'ticket_number': imputed_train_X['ticket_number'],
    'PassengerId': imputed_train_X['PassengerId'],
    'Survived': train_Y
})

In [ ]:
correlation_df['ticket_number'].isna().sum()

4

In [ ]:
correlation_df = correlation_df.dropna(subset=['ticket_number'])

In [ ]:
correlation_df.isna().sum()

ticket_number    0
PassengerId      0
Survived         0
dtype: int64

In [ ]:
correlation_df['ticket_number'] = correlation_df['ticket_number'].astype(int)

In [ ]:
from scipy.stats import pointbiserialr
TN_corr, _ = pointbiserialr(correlation_df['ticket_number'], correlation_df['Survived'])
PI_corr, _ = pointbiserialr(correlation_df['PassengerId'], correlation_df['Survived'])
print(TN_corr, PI_corr)

-0.06481569394043273 0.020176915738406426


Since the correlation value for ticket number and passenger id is low, we decided to drop these two columns.

In [ ]:
imputed_train_X = imputed_train_X.drop(columns=['ticket_number', 'PassengerId'])
imputed_val_X = imputed_val_X.drop(columns=['ticket_number', 'PassengerId'])
imputed_test_X = imputed_test_X.drop(columns=['ticket_number', 'PassengerId'])

In [ ]:
imputed_train_X

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,family_name,title,ticket_prefix
331,1,"Partner, Mr. Austen",male,45.5,0,0,113043,28.5000,S,Partner,Mr.,None
733,2,"Berriman, Mr. William John",male,23.0,0,0,28425,13.0000,S,Berriman,Mr.,None
382,3,"Tikkanen, Mr. Juho",male,32.0,0,0,STON/O 2. 3101293,7.9250,S,Tikkanen,Mr.,STON/O 2.
704,3,"Hansen, Mr. Henrik Juul",male,26.0,1,0,350025,7.8542,S,Hansen,Mr.,None
813,3,"Andersson, Miss. Ebba Iris Alfrida",female,6.0,4,2,347082,31.2750,S,Andersson,Miss.,None
...,...,...,...,...,...,...,...,...,...,...,...,...
106,3,"Salkjelsvik, Miss. Anna Kristine",female,21.0,0,0,343120,7.6500,S,Salkjelsvik,Miss.,None
270,1,"Cairns, Mr. Alexander",male,28.0,0,0,113798,31.0000,S,Cairns,Mr.,None
860,3,"Hansen, Mr. Claus Peter",male,41.0,2,0,350026,14.1083,S,Hansen,Mr.,None
435,1,"Carter, Miss. Lucile Polk",female,14.0,1,2,113760,120.0000,S,Carter,Miss.,None


In [ ]:
imputed_train_X.drop(['Name', 'Ticket'], axis=1, inplace=True)
imputed_val_X.drop(['Name', 'Ticket'], axis=1, inplace=True)
imputed_test_X.drop(['Name', 'Ticket'], axis=1, inplace=True)

In [ ]:
imputed_train_X.to_csv('../data/curated/cleaned_train_X.csv', index=False)
imputed_val_X.to_csv('../data/curated/cleaned_val_X.csv', index=False)
imputed_test_X.to_csv('../data/curated/cleaned_test_X.csv', index=False)
train_Y.to_csv('../data/curated/train_Y.csv', index=False)
val_Y.to_csv('../data/curated/val_Y.csv', index=False)